## Perform traditional bulk pycytominer pipeline

Following single-cell curation with cytotable, we create bulk profiles by applying the following steps:

1. aggregation
2. annotation
3. normalization
4. feature_selection

In [1]:
import pathlib
import yaml
import pprint

import pandas as pd

from pycytominer import aggregate, annotate, normalize, feature_select
from pycytominer.cyto_utils import load_profiles, output

In [2]:
# Set constants
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
    "drop_na_columns",
]

# Set paths
output_dir = pathlib.Path("data/bulk_profiles")
output_dir.mkdir(exist_ok=True)
metadata_dir = pathlib.Path("../0.download_data/metadata/")

# load in plate information
dictionary_path = pathlib.Path("./plate_info_dictionary.yaml")
with open(dictionary_path) as file:
    plate_info_dictionary = yaml.load(file, Loader=yaml.FullLoader)

In [3]:
# add path to platemaps for each plate
for plate in plate_info_dictionary.keys():
    # since Plate_3_prime has the same platemap as Plate_3,
    # we need an else statement so that we make sure it adds the
    # path that was given to Plate_3
    if plate != "Plate_3_prime":
        # match the naming format of the plates to the platemap file
        plate_info_dictionary[plate]["platemap_path"] = str(
            pathlib.Path(
                list(
                    metadata_dir.rglob(
                        f"platemap_NF1_{plate.replace('_', '').lower()}.csv"
                    )
                )[0]
            ).resolve(strict=True)
        )
    else:
        plate_info_dictionary["Plate_3_prime"]["platemap_path"] = plate_info_dictionary[
            "Plate_3"
        ]["platemap_path"]

# view the dictionary to assess that all info is added correctly
pprint.pprint(plate_info_dictionary, indent=4)

{   'Plate_6': {   'dest_path': 'data/converted_data/Plate_6.parquet',
                   'platemap_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/0.download_data/metadata/platemap_NF1_plate6.csv',
                   'source_path': '/media/18tbdrive/1.Github_Repositories/nf1_schwann_cell_painting_data/2.cellprofiler_analysis/analysis_output/Plate_6/Plate_6_nf1_analysis.sqlite'}}


## Perform pycytominer pipeline

In [4]:
for plate, info in plate_info_dictionary.items():
    print(f"Now performing pycytominer pipeline for {plate}")
    output_aggregated_file = str(pathlib.Path(f"{output_dir}/{plate}_bulk.parquet"))
    output_annotated_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_annotated.parquet")
    )
    output_normalized_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_normalized.parquet")
    )
    output_feature_select_file = str(
        pathlib.Path(f"{output_dir}/{plate}_bulk_feature_selected.parquet")
    )

    # Load single-cell profiles
    single_cell_df = pd.read_parquet(info["dest_path"])

    # Load platemap
    platemap_df = pd.read_csv(info["platemap_path"])

    # Step 1: Aggregation
    aggregate(
        population_df=single_cell_df,
        operation="median",
        strata=["Image_Metadata_Plate", "Image_Metadata_Well"],
        output_file=output_aggregated_file,
        output_type="parquet",
    )

    # Step 2: Annotation
    annotated_df = annotate(
        profiles=output_aggregated_file,
        platemap=platemap_df,
        join_on=["Metadata_well_position", "Image_Metadata_Well"],
    )

    # For only plates 3 and 3 prime, remove any rows with HET due to contamination
    if plate in ["Plate_3", "Plate_3_prime"]:
        # Filter single-cell profiles, removing HET genotype
        annotated_df = annotated_df[annotated_df["Metadata_genotype"] != "HET"]
        print("HET cells have been removed from", plate)

    # use output to save annotated df as you can not use the output parameter in the annotate which will not return data frame
    output(
        df=annotated_df,
        output_filename=output_annotated_file,
        output_type="parquet",
    )

    # Step 3: Normalization
    normalized_df = normalize(
        profiles=annotated_df,
        method="standardize",
        output_file=output_normalized_file,
        output_type="parquet",
    )

    # Step 4: Feature selection
    feature_select(
        output_normalized_file,
        operation=feature_select_ops,
        na_cutoff=0,
        output_file=output_feature_select_file,
        output_type="parquet",
    )

Now performing pycytominer pipeline for Plate_6


In [5]:
# Check output file
test_df = load_profiles(output_feature_select_file)

print(test_df.shape)
test_df.head(2)

(60, 843)


Metadata_WellRow  Metadata_WellCol Metadata_gene_name Metadata_genotype  \
0                B                 2                NF1                WT   
1                B                 3                NF1                WT   

  Metadata_Institution   Metadata_seed_density  Metadata_Plate Metadata_Well  \
0              iNFixion                   1000             NaN            B2   
1              iNFixion                   1000             NaN            B3   

   Cytoplasm_AreaShape_BoundingBoxMaximum_X  \
0                                  0.797740   
1                                  1.565229   

   Cytoplasm_AreaShape_BoundingBoxMaximum_Y  ...  \
0                                  1.775785  ...   
1                                 -0.292374  ...   

   Nuclei_Texture_Correlation_RFP_3_02_256  \
0                                -0.057772   
1                                -0.530853   

   Nuclei_Texture_Correlation_RFP_3_03_256  \
0                                 0.293901   
1                                 0.515466   

   Nuclei_Texture_DifferenceVariance_DAPI_3_03_256  \
0                                        -0.091186   
1                                        -0.324181   

   Nuclei_Texture_InfoMeas1_DAPI_3_00_256  \
0                                0.681027   
1                                0.597365   

   Nuclei_Texture_InfoMeas1_GFP_3_02_256  \
0                               0.305274   
1                              -0.100962   

   Nuclei_Texture_InfoMeas1_RFP_3_03_256  \
0                              -0.684586   
1                              -1.312681   

   Nuclei_Texture_InfoMeas2_DAPI_3_02_256  \
0                               -0.328401   
1                               -0.655604   

   Nuclei_Texture_InfoMeas2_GFP_3_03_256  \
0                               0.260365   
1                               0.733149   

   Nuclei_Texture_InverseDifferenceMoment_DAPI_3_03_256  \
0                                           0.217471      
1                                           0.424428      

   Nuclei_Texture_InverseDifferenceMoment_GFP_3_01_256  
0                                          -0.093362    
1                                          -0.554693    

[2 rows x 843 columns]